# Metabolon data to KEGG gene mergeing

## Inputs:

- xlsx file from Metabolon, tab to use

Notes: 

    Biochemical Name*	Indicates compounds that have not been officially confirmed based on a standard, but we are confident in its identity.
    Biochemical Name**	Indicates a compound for which a standard is not available, but we are reasonably confident in its identity or the information provided.

    Biochemical Name (#) or [#]	Indicates a compound that is a structural isomer of another compound in the Metabolon spectral library.
        For example, a steroid that may be sulfated at one of several positions that are indistinguishable by the mass spectrometry data or a diacylglycerol for which more than one stereospecific molecule exists.



## Input data saved

- tsv files of sample metadata, compound/peak metadata, and abundance data

## Reference DBs:

- Web REST accesees current KEGG compound, reaction, ortholog datasets, and compound to reaction, reaction to KEGG ortholog data
- outputs raw and merged KEGG reference data used

## data handeling

- filter out very common reaction compounds (CAD(P)/H, A(D/T)P, PO4
- remove any unidentified cpounds, or those without a KEGG copound ID (drug IDs not used), or a ko
- use both in the case of duplicate differing KEGG IDs

## Outputs

- merged compound to keg metadata, and simple chemical ID to ko table



## Import raw metabolon data

In [ ]:
#libraries needed
import pandas as pd
import numpy as np

In [ ]:
#change settings here
#load tab from file (all data, multindex magles nulls)
infile='../test.xlsx'
infiletab='OrigScale'

cols_metadata=13
rows_metadata=27

In [ ]:
cols_metadata=int(cols_metadata)
rows_metadata=int(rows_metadata)

In [ ]:
#read file
rawOrigScale=pd.read_excel(infile, sheet_name=infiletab, header=None)

In [ ]:
#get sample meta data
sample_meta=rawOrigScale.iloc[:rows_metadata,cols_metadata:].T

#sample metadata column names
sample_meta_columns=rawOrigScale.iloc[:rows_metadata,cols_metadata-1].to_list()
#row and col metadata are mangles into one cell, split it to correct them
sample_meta_columns=sample_meta_columns[:-1]+[sample_meta_columns[-1].lstrip().split()[0]]

#add in column names
sample_meta.columns=sample_meta_columns

#drop redundant col and index with uniqID
assert (sample_meta.iloc[:,-1]==sample_meta['GROUP_DESCRIPTION']).all()
sample_meta = sample_meta.iloc[:,:-1]
sample_meta = sample_meta.set_index(sample_meta['CLIENT_IDENTIFIER'], verify_integrity=True)
sample_meta.to_csv('_'.join([infile,infiletab,'sampleMETADATA'])+'.tsv')

In [ ]:
#pull out metadata for compounds/peaks
chem_meta=rawOrigScale.iloc[rows_metadata:,:cols_metadata]

#compounds/peaks metadata column names
chem_meta_columns=rawOrigScale.iloc[rows_metadata-1,:cols_metadata].to_list()
#row and col metadata are mangles into one cell, split it to correct them
chem_meta_columns=chem_meta_columns[:-1]+[chem_meta_columns[-1].lstrip().split()[1]]

#add in column names
chem_meta.columns=chem_meta_columns

chem_meta = chem_meta.set_index(chem_meta['BIOCHEMICAL'], verify_integrity=True)
sample_meta.to_csv('_'.join([infile,infiletab,'metabolitesMETADATA'])+'.tsv')

In [ ]:
#get data
rawdata=rawOrigScale.iloc[rows_metadata:,cols_metadata:].astype(np.float64)
metabolomics=rawdata.set_axis(chem_meta.index, axis=0).set_axis(sample_meta.index, axis=1)
sample_meta.to_csv('_'.join([infile,infiletab,'occurance'])+'.tsv')
metabolomics


In [ ]:
#how many do we have compounds for?
#should be 958+4 (at end) lines, then 1270-(958+4) without IDs

#now many do we have DB iDs for?
print('number of peaks\t\t%s\n' % len(chem_meta))
for DB in 'CHEMICAL_ID	COMP_ID	PUBCHEM	CAS	KEGG	HMDB'.split():
        #print( len(chem_meta[DB])-sum(chem_meta[DB].isna())
        print(chem_meta[DB].describe())
        print()

In [ ]:
print("percent of peaks with the following IDs")
nulls=chem_meta['CHEMICAL_ID	PUBCHEM	CAS	KEGG	HMDB'.split()].isnull().sum()
(100.0*(len(chem_meta[DB])-nulls)/len(chem_meta[DB])).round(1).sort_values(ascending=False)


In [ ]:
chem_meta['SUPER_PATHWAY'].unique()

In [ ]:
chem_meta['SUB_PATHWAY'].unique()

In [ ]:
chem_meta.head()

In [ ]:
#searching for strings 
s='methylamine'
chem_meta['BIOCHEMICAL'][chem_meta['BIOCHEMICAL'].str.match(r'.*'+s+'.*', case=False)]

In [ ]:
chem_meta[chem_meta.BIOCHEMICAL.str.match(r'.*'+s+'.*', case=False)].T

## Retreve all KEGG reactions

See: https://www.kegg.jp/kegg/rest/keggapi.html

Kinds of links avaliable for compounds:

https://www.genome.jp/dbget-bin/get_linkdb?targettype=all&keywords=cpd%3AC00025&targetformat=html&targetdb=alldb



### get all KEGG IDS as tsv files


In [ ]:
%%bash

mkdir -p kegg

curl http://rest.kegg.jp/info/compound | tee kegg/compound.info
wget -P kegg http://rest.kegg.jp/list/compound
    curl http://rest.kegg.jp/info/reaction | tee kegg/reaction.info
wget -P kegg http://rest.kegg.jp/list/reaction
    curl http://rest.kegg.jp/info/ko | tee kegg/ko.info
wget -P kegg http://rest.kegg.jp/list/ko
    
wget http://rest.kegg.jp/link/compound/reaction -O kegg/compound-reaction
wget http://rest.kegg.jp/link/reaction/ko -O kegg/reaction-ko


In [ ]:
!wc -l kegg/*

### Example information avaliable per entry:

http://rest.kegg.jp/get/cpd:C00025

    ENTRY       C00025                      Compound
    ...
    REACTION    R00021 R00093 R00114 R00239 R00241 R00243 R00245 R00248 
                R00250 R00251 R00253 R00254 R00256 R00257 R00258 R00259 
                ...
    ...

http://rest.kegg.jp/get/rn:R00519

    ENTRY       R00519                      Reaction
    ...
    ORTHOLOGY   K00122  formate dehydrogenase [EC:1.17.1.9]
                K00123  formate dehydrogenase major subunit [EC:1.17.1.9]
                K00124  formate dehydrogenase iron-sulfur subunit
                K00126  formate dehydrogenase subunit delta [EC:1.17.1.9]
                K00127  formate dehydrogenase subunit gamma
                K22515  formate dehydrogenase beta subunit [EC:1.17.1.9]
                ...
    ...



In [ ]:
#fuinctions to get and parse KEGG

def getkid(ser):
    h=ser.str.split(':').str[0].unique()
    assert len(h)==1
    return h[0]

def kidprefix_to_colname(df):
    df.columns=df.apply(getkid)
    return df.apply(lambda x: x.str.split(':').str[1])

def KEGG_retrieve(dbname):
    url='http://rest.kegg.jp/list/'+dbname
    print('downloading %s' % url)
    df=pd.read_csv(url, sep='\t', header=None)
    #get kid out of first element, put in column headers
    listtkid=getkid(df[0])
    df.columns=[listtkid, listtkid+'_description']
    #remove kid from first col
    df[listtkid]=df[listtkid].str.split(':').str[1]
    df.columns.name=dbname
    return df

def KEGG_link(db1,db2):
    url='http://rest.kegg.jp/link/'+db1+'/'+db2
    print('downloading %s' % url)
    df=pd.read_csv(url, sep='\t', header=None)
    df.columns.name=db1+'_'+db2
    return kidprefix_to_colname(df)

Use REST interface to get the same file as above. 

In [ ]:
%%time
c=KEGG_retrieve('compound')
r=KEGG_retrieve('reaction')
k=KEGG_retrieve('ko')

cr=KEGG_link('compound','reaction')
rk=KEGG_link('reaction','ko')


In [ ]:
kegg_crk = pd.merge(cr,rk, how='outer', indicator='merged_reactionlinks')
kegg_crk.head()

In [ ]:
kegg_crk.describe()

In [ ]:
kegg_crk.merged_reactionlinks.value_counts()

In [ ]:
#add in desc
for d in c, r, k:
    mergeind='merged_'+d.columns.name
    kegg_crk = pd.merge(kegg_crk,d, how='outer', indicator=mergeind)
    print(kegg_crk[mergeind].value_counts())

kegg_crk.head()

In [ ]:
kegg_crk.describe()

In [ ]:
kegg_crk=kegg_crk.drop(kegg_crk.columns[kegg_crk.columns.str.match(r'merged_.*')], axis=1)


In [ ]:
kegg_crk.to_csv('kegg/merged_crk.tsv', sep='\t')

## pull out uniq compounds from metabolon and link to ko to exsplore data

In [ ]:
#pull out unique CID for KEGG lookup
metabolonKEGG=pd.Series(chem_meta['KEGG'].dropna().unique())
            
#occasionally there is a drug / compound cross-ref, er wna tthe compound
def getCID(KEGGstr):
    ids=[c for c in KEGGstr.split(',') if c[0]=='C']
    if len(ids)==1: return ids[0]
    else: return KEGGstr

#drop dupes again in case the drug ones were dupes
metabolonKEGG=metabolonKEGG.apply(getCID).dropna().unique()
metabolonKEGG=pd.DataFrame(metabolonKEGG, columns=['cpd'])

In [ ]:
metabolon_crk=pd.merge(metabolonKEGG, kegg_crk, how='left', indicator='merged')

In [ ]:
print(len(metabolon_crk))
metabolon_crk.describe()

### common cpd are not usefull to trace into rxn and genes

In [ ]:
for desc in metabolon_crk.columns[metabolon_crk.columns.str.match(r'.*_description')]:
    print('\n'+desc)
    print(metabolon_crk[desc].value_counts(ascending=False).head(20))

In [ ]:
#common cpd are not usefull to trace into rxn and genes
metabolon_crk['cpd'].value_counts(ascending=False).head(20)

In [ ]:
#drop ATP, ADP, NAD, NADH, Phosphate, ect
c.head(17)
    

In [ ]:
fromcommon=metabolon_crk[metabolon_crk['cpd'].isin(c.head(17)['cpd'])]
print('WARNING: these very common cofactors are found in the dataset')
fromcommon['cpd_description'].value_counts()

### remerge

In [ ]:
metabolon_crk=pd.merge(metabolonKEGG, kegg_crk, how='left', indicator='merged')

print('WARNING: dropping H2O, NAD(P)/H, ATP/ADP, and PO4')
print('was %s lines' % len(metabolon_crk))

droplist=set('''C00001
C00002
C00003
C00004
C00005
C00006
C00008
C00009'''.split())

#merge just like above, but remove the common cofactors from metabolon list first, so as not to pull in junk
metabolon_crk=pd.merge(metabolonKEGG[~metabolonKEGG['cpd'].isin(droplist)], kegg_crk, how='left', indicator='merged')

#metabolon_crk=metabolon_crk[~metabolon_crk.cpd.isin(droplist)]

print('now %s lines' % len(metabolon_crk))
metabolon_crk.describe()



In [ ]:
#drop those without any ko genes linked to these compounts
metabolon_crk=metabolon_crk.drop('merged', axis=1).dropna(subset=['ko']).drop_duplicates()
metabolon_crk

In [ ]:
#get the unique othologs involved
metabolon_ko = metabolon_crk[['ko','ko_description']].dropna(subset=['ko']).drop_duplicates()
metabolon_ko

In [ ]:
metabolon_ko.describe()

# Merge Metabolon with KEGG ortholog data

In [ ]:
#there are duplicate KEGGs in the metabolon table, we will let these duplicate data, as at leasr some seen different. 
tmp=chem_meta[~chem_meta['KEGG'].isin(droplist)].dropna(subset=['KEGG']).drop_duplicates()
tmp[tmp.duplicated(subset=['KEGG'], keep=False)]

In [ ]:
#clean for merge
print('WARNING: dropping H2O, NAD(P)/H, ATP/ADP, and PO4')
droplist=set('''C00001
C00002
C00003
C00004
C00005
C00006
C00008
C00009'''.split())

chem_meta_clean=chem_meta[~chem_meta['KEGG'].isin(droplist)].dropna(subset=['KEGG']).drop_duplicates()
kegg_crk_clean=kegg_crk[~kegg_crk['cpd'].isin(droplist)].dropna(subset=['cpd']).drop_duplicates()

chem_meta_clean_ko=chem_meta_clean.merge(kegg_crk_clean, how='left', indicator='merged', left_on='KEGG', right_on='cpd')
chem_meta_clean_ko
 

In [ ]:
chem_meta_clean_ko.describe()

In [ ]:
chem_meta_clean_ko[chem_meta_clean_ko['merged']=='left_only']

In [ ]:
chem_meta_clean_ko=chem_meta_clean_ko.drop('merged', axis=1).dropna(subset=['CHEMICAL_ID','KEGG','ko'])
chem_meta_clean_ko.to_csv('_'.join([infile,infiletab,'metabolitesMETADATA_filtered_KEGGmerge'])+'.tsv')
chem_meta_clean_ko.describe()

In [ ]:
#get simple CHEMICAL_ID	COMP_ID to ko table
chem_ko=chem_meta_clean_ko.copy(deep=True)
chem_ko=chem_ko[['CHEMICAL_ID','KEGG','ko']].dropna(how='any').drop_duplicates()
chem_ko

In [ ]:
chem_ko.to_csv('_'.join([infile,infiletab,'metabolitesCHEMICALID_filtered_KEGG_ko'])+'.tsv')
chem_ko.describe()

## quick test comparison to ko found in metagenomics Novogene annotations

In [ ]:
!rsync jamesrh@cvmrit03.cvmbs.colostate.edu:/lab_data/ryan_lab/jamesrh/ERyan_Novogene_contract_H202SC19111892_working/result/05.FunctionAnnotation/KEGG/GeneNums/Unigenes.absolute.ko.xls ./
!cat Unigenes.absolute.ko.xls  | cut -f 1-4 >> tmp.Unigenes.absolute.ko.xls 

In [ ]:
MG_KEGG=pd.read_csv('tmp.Unigenes.absolute.ko.xls', sep='\t')


In [ ]:
MG_KEGG

In [ ]:
test = chem_meta_clean_ko.merge(MG_KEGG, how='outer', left_on='ko', right_on='KO_ID', indicator='merged')

In [ ]:
print(len(test))
test.describe()

In [ ]:
t=test.merged.value_counts().to_frame()

In [ ]:
test.merged.value_counts().to_frame()

In [ ]:
for v in test.merged.value_counts().to_frame().iterrows():
    print("\n%s total %s\n" % (v[0], v[1][0]))
    print(test[test['merged']==v[0]]['SUPER_PATHWAY'].value_counts(dropna=False))

c.cpd.value_counts()

In [ ]:
test2=test[test['merged']=='both'][['SUPER_PATHWAY','SUB_PATHWAY','rn_description','BIOCHEMICAL']]
test2=test2.groupby(['SUPER_PATHWAY','SUB_PATHWAY']).agg({"rn_description": pd.Series.nunique, "BIOCHEMICAL": pd.Series.nunique})
test2

In [ ]:
#only rarely are there multiple chem per rxn
test2[test2['rn_description']/test2['BIOCHEMICAL']<1]


In [ ]:
#usually, there are more rxn then chemicals
test2[test2['rn_description']/test2['BIOCHEMICAL']>=1]